In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
10.100.212.75
ext-pw-faksana
Y685iQq0IqItwQ

In [2]:
def extract_data(path):
    ds = xr.open_dataset(path,engine='cfgrib')
    df = ds.to_dataframe().reset_index()
    return ds, df


def check_nan(df,month, year):
    total= df.shape[0]
    false = df.isnull().any(axis=1).value_counts().tolist()[0]
    true = df.isnull().any(axis=1).value_counts().tolist()[1] 
    # print(f'total rows is {total}')
    print(f'the data in {month} {year}')
    print(f'missing value is {round(true/total * 100)} %')
    print(f'availabel value is {round(false/total * 100)} %')
    print('==========================')
    
def check_null_by_coor(df):
    cols_to_check = ['d2m', 't2m', 'ssr', 'ssrd', 'u10', 'v10', 'sp']
    lat_deleted = []
    lon_deleted = []

    # FIX: Only loop through UNIQUE coordinates to avoid millions of redundant checks
    unique_lats = df['latitude'].unique()
    unique_lons = df['longitude'].unique()

    for lat in unique_lats:
        for lon in unique_lons:
            # Filter the dataframe for the specific coordinate
            mask = (df['latitude'] == lat) & (df['longitude'] == lon)
            subset = df[mask][cols_to_check]

            if subset.empty:
                continue 

            # Check for missing values across rows
            missing_mask = subset.isnull().any(axis=1)
            
            # Use .sum() for booleans; it's faster than value_counts()
            num_missing = missing_mask.sum()
            total_rows = len(subset)

            if num_missing > 0:
                missing_ratio = num_missing / total_rows
                
                if missing_ratio > 0.7:
                    #print(f"CRITICAL: Lat {lat}, Lon {lon} has {missing_ratio:.2%} missing data.")
                    lat_deleted.append(lat)
                    lon_deleted.append(lon)
            else:
                # Optional: This might spam your console if you have many coordinates
                # print(f"No missing values for Lat: {lat}, Lon: {lon}")
                pass

    coor_del = pd.DataFrame({'latitude': lat_deleted, 'longitude': lon_deleted})
    # summary = coor_del.groupby('lat').agg({'lon': ['count']})
    # print(summary)
    return coor_del

In [3]:
def plot_coor(df):
    fig = px.scatter_mapbox(
        df,
        lat="latitude",
        lon="longitude",
        zoom=6,
        height=600
    )

    fig.update_layout(
        mapbox_style="open-street-map"
    )
    fig.show()

In [4]:
# _, jan_df = extract_data('data/january_2022.grib')
# _, feb_df = extract_data('data/february_2022.grib')
# _, mar_df = extract_data('data/march_2022.grib')
# _, apr_df = extract_data('data/april_2022.grib')
# _, may_df = extract_data('data/may_2022.grib')
# _, jun_df = extract_data('data/june_2022.grib')

# check_nan(jan_df,'january',2022)
# check_nan(feb_df,'february',2022)

# coor_del_jan22 = check_null_by_coor(jan22_df)
# coor_del_feb22 = check_null_by_coor(feb22_df)

# with pd.ExcelWriter('coordinate_deleted.xlsx') as writer:
#     coor_del_jan22.to_excel(writer, sheet_name='January_2022', index=False)
#     coor_del_feb22.to_excel(writer, sheet_name='February_2022', index=False)

In [ ]:
# plot_coor(jan22_df[['latitude', 'longitude']].drop_duplicates())
# cols_to_check = ['d2m', 't2m', 'ssr', 'ssrd', 'u10', 'v10', 'sp']
# jan22_df_clean = jan22_df.dropna(subset=cols_to_check )
# plot_coor(jan22_df_clean[['latitude', 'longitude']].drop_duplicates())

#### Clean data

In [5]:
def extract_data_clean(path):
    ds = xr.open_dataset(path,engine='cfgrib')
    df = ds.to_dataframe().reset_index()
    cols_to_check = ['d2m', 't2m', 'ssr', 'ssrd', 'u10', 'v10', 'sp']
    df_clean = df.dropna(subset=cols_to_check )
    return df_clean

In [6]:
jan_df = extract_data_clean('data/january_2022.grib')
feb_df = extract_data_clean('data/february_2022.grib')
mar_df = extract_data_clean('data/march_2022.grib')
apr_df = extract_data_clean('data/april_2022.grib')
may_df = extract_data_clean('data/may_2022.grib')
jun_df = extract_data_clean('data/june_2022.grib')

In [7]:
year_df = pd.concat([jan_df, feb_df, mar_df, apr_df, may_df, jun_df], ignore_index=True)

In [8]:

df_sorted = year_df.sort_values(
    by=["latitude", "longitude", "valid_time"]
).reset_index(drop=True)

df_sorted["valid_time"] = pd.to_datetime(df_sorted["valid_time"])

In [9]:
def get_missing_times(group):
    expected_range = pd.date_range(
        start=group["valid_time"].min(),
        end=group["valid_time"].max(),
        freq="1h"
    )
    
    missing_times = expected_range.difference(group["valid_time"])
    
    if len(missing_times) > 0:
        return pd.DataFrame({
            "latitude": group.name[0],
            "longitude": group.name[1],
            "missing_time": missing_times
        })

missing_details = (
    df_sorted.groupby(["latitude", "longitude"])
      .apply(get_missing_times)
      .dropna()
      .reset_index(drop=True)
)

missing_details

,latitude,longitude,missing_time
0,49.3,14.57,2022-04-03 05:00:00
1,49.3,23.17,2022-05-24 03:00:00
2,49.7,20.97,2022-06-28 03:00:00
3,49.7,21.27,2022-04-18 04:00:00
4,49.7,22.07,2022-03-19 05:00:00
5,49.7,23.67,2022-05-26 03:00:00
6,50.1,23.07,2022-05-27 03:00:00
7,50.1,23.17,2022-05-27 03:00:00
8,50.1,24.07,2022-05-22 03:00:00
9,50.2,20.07,2022-06-21 03:00:00


In [12]:
def fill_missing_hours(group):

    group = group.copy()

    # Get coordinate values from group index
    lat_val = group["latitude"].iloc[0]
    lon_val = group["longitude"].iloc[0]

    # Create full hourly time grid
    full_time = pd.date_range(
        start=group["valid_time"].min(),
        end=group["valid_time"].max(),
        freq="1h"
    )

    # Set time index
    group = group.set_index("valid_time")

    # Reindex
    group = group.reindex(full_time)

    # Restore coordinate columns ⭐
    group["latitude"] = lat_val
    group["longitude"] = lon_val

    # Forward fill other variables
    group = group.ffill()

    # Reset index
    group = group.rename_axis("valid_time").reset_index()

    return group

In [14]:
df_filled = (
    df_sorted.groupby(["latitude", "longitude"], group_keys=False)
    .apply(fill_missing_hours)
    .reset_index(drop=True)
)

KeyError: 'latitude'

In [ ]:
df_filled

In [ ]:
df_sorted

In [ ]:
# df_sorted.to_csv('data_sorted.csv')

In [ ]:


df_filtered = df_sorted[
    np.isclose(df_sorted["latitude"], 49.3) &
    np.isclose(df_sorted["longitude"], 14.57) &
    (df_sorted["valid_time"] >= "2022-04-03 01:00:00") &
    (df_sorted["valid_time"] <= "2022-04-03 08:00:00")
]

In [ ]:
df_filtered

In [ ]:
df_filtered = df_filled[
    np.isclose(df_filled["latitude"], 49.3) &
    np.isclose(df_filled["longitude"], 14.57) &
    (df_filled["valid_time"] >= "2022-04-03 01:00:00") &
    (df_filled["valid_time"] <= "2022-04-03 08:00:00")
]
df_filtered

In [ ]:
df_filled